In [1]:
import numpy as np
import pandas as pd
import cv2

import keras.layers as kr

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
train = pd.read_csv('train.csv')

In [4]:
grp = train.groupby(['x','y'])

In [9]:
grp.any()

,,id
x,y,
-171.180678,73.78272,True
-163.780678,78.38272,True
-158.680678,22.68272,True
-158.380678,19.78272,True
-158.180678,81.98272,True
...,...,...
86.719322,-24.31728,True
87.319322,-26.71728,True
88.619322,-28.81728,True
